# Hotel Revenue MONTHLY Time Series Analysis - EDA & Preprocessing
## Multi-Year Forecasting for 2025 Annual Close

**Project Goal:** Predict hotel revenue metrics for September - December 2025 using historical **MONTHLY data from 2009-2025**.

**Current Date:** August 2025

**Data Splits (MONTHLY):**
- Training: 2009-01 to 2025-05 (16+ years of history!)
- Validation: 2025-06 to 2025-08 (3 months)
- Test/Forecast: 2025-09 to 2025-12 (4 months)

---

## Section 1: Setup and Data Loading

### 1.1 Libraries Import

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Statistical libraries for time series
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

# Scikit-learn utilities
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries imported successfully!")

### 1.2 Configuration and Constants

In [ ]:
# Define key constants
CURRENT_DATE = '2025-08'  # MONTHLY format
TARGET_VARIABLES = ['RevPar', 'ADR', 'Revenue', 'Occupancy_Pct']

# Data split dates (MONTHLY FORMAT)
TRAIN_END = '2025-05'  # May 2025
VALIDATION_START = '2025-06'  # June 2025
VALIDATION_END = '2025-08'  # August 2025
FORECAST_START = '2025-09'  # September 2025
FORECAST_END = '2025-12'  # December 2025

# Color palette for visualizations
COLORS = sns.color_palette('husl', 8)

print(f"Configuration set:")
print(f"  - Current Date: {CURRENT_DATE}")
print(f"  - Training Period: 2009-01 to {TRAIN_END}")
print(f"  - Validation Period: {VALIDATION_START} to {VALIDATION_END}")
print(f"  - Forecast Period: {FORECAST_START} to {FORECAST_END}")
print(f"  - Target Variables: {TARGET_VARIABLES}")

### 1.3 Data Ingestion (Google Colab File Upload)

**Instructions:** Upload your `monthly_revenue_2009_2025_all.csv` file when prompted below.

In [ ]:
# Google Colab file upload
from google.colab import files

print("Please upload your monthly_revenue_2009_2025_all.csv file:")
uploaded = files.upload()

# Get the uploaded filename
filename = list(uploaded.keys())[0]
print(f"\nFile '{filename}' uploaded successfully!")

In [ ]:
# Load the data
df = pd.read_csv(filename)

# Create Date column from Year and Month columns
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2) + '-01')

# Rename columns to match notebook expectations
# CSV columns: Year,Month,Month_Name,Days,Total_Revenue,Total_Rooms_Sold,ADR,Occupancy_Pct,RevPar
# Expected: Date, Revenue, ADR, RevPar, Occupancy_Pct
df = df.rename(columns={
    'Total_Revenue': 'Revenue',
    'Total_Rooms_Sold': 'Rooms_Sold'
})

# Sort by date
df = df.sort_values('Date').reset_index(drop=True)

print(f"Data loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Columns: {list(df.columns)}")

---
## Section 2: Exploratory Data Analysis (EDA)

### 2.1 Initial Data Audit

In [ ]:
# Display basic information
print("="*80)
print("DATASET INFORMATION")
print("="*80)
print(f"\nDataset shape: {df.shape}")
print(f"Number of records: {len(df)}")
print(f"Number of features: {len(df.columns)}")
print(f"\nColumn names:\n{list(df.columns)}")

In [ ]:
# Data types and memory usage
print("\n" + "="*80)
print("DATA TYPES AND MEMORY")
print("="*80)
df.info()

In [ ]:
# First and last records
print("\n" + "="*80)
print("FIRST 5 RECORDS")
print("="*80)
display(df.head())

print("\n" + "="*80)
print("LAST 5 RECORDS")
print("="*80)
display(df.tail())

In [ ]:
# Summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
display(df.describe())

In [ ]:
# Check for missing values
print("\n" + "="*80)
print("MISSING VALUES CHECK")
print("="*80)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing Count': missing.values,
    'Percentage': missing_pct.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    display(missing_df)
else:
    print("No missing values found!")

In [ ]:
# Check for duplicates
print("\n" + "="*80)
print("DUPLICATE CHECK")
print("="*80)
duplicates = df.duplicated(subset='Date').sum()
print(f"Number of duplicate dates: {duplicates}")

if duplicates > 0:
    print("\nDuplicate dates:")
    display(df[df.duplicated(subset='Date', keep=False)].sort_values('Date'))

### 2.2 Time Series Visualization

In [ ]:
# Plot all target variables over time
fig, axes = plt.subplots(4, 1, figsize=(16, 12))

for idx, var in enumerate(TARGET_VARIABLES):
    axes[idx].plot(df['Date'], df[var], linewidth=1, color=COLORS[idx])
    axes[idx].set_title(f'{var} Over Time (2009-2025)', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=11)
    axes[idx].set_ylabel(var, fontsize=11)
    axes[idx].grid(True, alpha=0.3)
    
    # Add vertical lines for data splits
    axes[idx].axvline(x=pd.to_datetime(TRAIN_END + '-01'), color='blue', linestyle='--', 
                      linewidth=1.5, label='Train End', alpha=0.7)
    axes[idx].axvline(x=pd.to_datetime(VALIDATION_END + '-01'), color='red', linestyle='--', 
                      linewidth=1.5, label='Current Date', alpha=0.7)
    axes[idx].legend()

plt.tight_layout()
plt.show()

### 2.3 Seasonality Analysis - Dubai Tourism Seasons

**Note:** Day-of-week analysis not applicable for monthly data. Analyzing monthly patterns instead.

In [ ]:
# Day-of-week analysis REMOVED - not applicable for monthly data
# This cell intentionally left blank for monthly analysis
print("✓ Day-of-week analysis skipped (not applicable for monthly granularity)")

In [ ]:
# Box plots by Month (across all years - showing Dubai seasonality)
df['Month'] = df['Date'].dt.month
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.ravel()

# Filter to only use data up to current date for analysis
df_for_analysis = df[df['Date'] <= pd.to_datetime(CURRENT_DATE + '-01')]

for idx, var in enumerate(TARGET_VARIABLES):
    sns.boxplot(data=df_for_analysis, x='Month', y=var, 
                palette='viridis', ax=axes[idx])
    axes[idx].set_title(f'{var} Distribution by Month (2009-2025)', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Month', fontsize=11)
    axes[idx].set_ylabel(var, fontsize=11)
    axes[idx].set_xticklabels(month_names)
    
    # Highlight Dubai seasons with background shading
    # High Season: Oct-Apr (months 10,11,12,1,2,3,4)
    # Low Season: May-Sep (months 5,6,7,8,9)
    axes[idx].axvspan(-0.5, 3.5, alpha=0.1, color='green', label='High Season (Oct-Apr)')
    axes[idx].axvspan(9.5, 11.5, alpha=0.1, color='green')
    axes[idx].axvspan(4.5, 8.5, alpha=0.1, color='red', label='Low Season (May-Sep)')
    axes[idx].legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.show()

### 2.4 Time Series Decomposition

In [ ]:
# Decompose RevPar time series (only training data)
train_eda = df[df['Date'] <= TRAIN_END].copy().set_index('Date')

decomposition = seasonal_decompose(train_eda['RevPar'], model='additive', period=12)

fig, axes = plt.subplots(4, 1, figsize=(16, 12))

decomposition.observed.plot(ax=axes[0], color='blue', marker='o')
axes[0].set_ylabel('Observed', fontsize=11)
axes[0].set_title('Time Series Decomposition - RevPar MONTHLY (Training Data)', fontsize=14, fontweight='bold')

decomposition.trend.plot(ax=axes[1], color='green', marker='o')
axes[1].set_ylabel('Trend', fontsize=11)

decomposition.seasonal.plot(ax=axes[2], color='orange')
axes[2].set_ylabel('Seasonal (12-month)', fontsize=11)

decomposition.resid.plot(ax=axes[3], color='red', marker='o')
axes[3].set_ylabel('Residual', fontsize=11)
axes[3].set_xlabel('Date', fontsize=11)

plt.tight_layout()
plt.show()

### 2.5 Autocorrelation Analysis

In [ ]:
# ACF and PACF for RevPar
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

plot_acf(train_eda['RevPar'].dropna(), lags=24, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF) - RevPar MONTHLY', fontsize=14, fontweight='bold')

plot_pacf(train_eda['RevPar'].dropna(), lags=24, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF) - RevPar MONTHLY', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 2.6 Stationarity Test

In [ ]:
# Augmented Dickey-Fuller Test
def adf_test(series, name=''):
    result = adfuller(series.dropna())
    print(f"\nAugmented Dickey-Fuller Test - {name}")
    print("="*60)
    print(f"ADF Statistic: {result[0]:.6f}")
    print(f"p-value: {result[1]:.6f}")
    print(f"Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.3f}")
    
    if result[1] <= 0.05:
        print(f"\nResult: STATIONARY (p-value = {result[1]:.6f})")
    else:
        print(f"\nResult: NON-STATIONARY (p-value = {result[1]:.6f})")

for var in TARGET_VARIABLES:
    adf_test(train_eda[var], name=var)

---
## Section 3: Feature Engineering and Preprocessing

Transform raw time series data into ML-ready features.

### 3.1 Data Cleaning

In [ ]:
# Create clean copy
df_clean = df.copy()

print("="*80)
print("DATA CLEANING (MONTHLY)")
print("="*80)

# Handle missing values using interpolation (monthly)
numeric_cols = ['Revenue', 'ADR', 'RevPar', 'Occupancy_Pct']
missing_before = df_clean[numeric_cols].isnull().sum().sum()

for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].interpolate(method='linear')  # Linear interpolation for monthly gaps
        # Fill any remaining NaN at the start/end with forward/backward fill
        df_clean[col] = df_clean[col].fillna(method='ffill').fillna(method='bfill')

missing_after = df_clean[numeric_cols].isnull().sum().sum()
print(f"Missing values before: {missing_before}")
print(f"Missing values after: {missing_after}")
print("\n✓ Data cleaning complete (monthly interpolation used)")

### 3.2 Temporal Feature Engineering

In [ ]:
print("\n" + "="*80)
print("TEMPORAL FEATURES (MONTHLY - OPTIMIZED, NO QUARTER)")
print("="*80)

# Create MINIMAL temporal features for monthly data
df_clean['Month'] = df_clean['Date'].dt.month  # 1-12 (for reference only)

# Dubai Hotel Seasonality (CRITICAL FEATURES - NO QUARTER!)
df_clean['High_Season'] = df_clean['Month'].isin([10,11,12,1,2,3,4]).astype(int)  # Oct-Apr (high tourism)
df_clean['Low_Season'] = df_clean['Month'].isin([5,6,7,8,9]).astype(int)  # May-Sep (low tourism)

print("✓ Created 2 temporal features (OPTIMIZED - NO QUARTER):")
print("  - High_Season (Oct-Apr) - Dubai tourism high season")
print("  - Low_Season (May-Sep) - Dubai tourism low season")
print("\n✓ REMOVED: Quarter (redundant with High/Low Season)")
print("  Total: 2 features (minimized for ~200 monthly records)")

### 3.3 Lagged Features

In [ ]:
print("\n" + "="*80)
print("LAGGED FEATURES (MONTHLY)")
print("="*80)

lag_features = ['RevPar', 'ADR', 'Occupancy_Pct', 'Revenue']
lag_periods = [1]  # 1-MONTH lag (not 1-day!)

for feature in lag_features:
    for lag in lag_periods:
        df_clean[f'{feature}_lag_{lag}'] = df_clean[feature].shift(lag)

print(f"✓ Created {len(lag_features) * len(lag_periods)} lagged features")
print(f"  - Variables: {lag_features}")
print(f"  - Lags: {lag_periods} MONTH (1-month lag)")
print(f"  - Note: Using only 1-month lag to avoid overfitting (~200 monthly records)")

### 3.4 Moving Averages and Rolling Statistics

In [ ]:
print("\n" + "="*80)
print("ROLLING STATISTICS")
print("="*80)

# IMPROVED: Remove MA/Rolling features for XGBoost to reduce multicollinearity
# Tree-based models like XGBoost automatically capture these patterns
# Keep these features commented out in case you use LSTM or SARIMA

print("✓ Skipping rolling features (MA, Std Dev) for XGBoost")
print("  - Reason: XGBoost captures these patterns automatically")
print("  - Reduces multicollinearity with lag features")
print("  - If using LSTM/SARIMA, uncomment the code below:")
print()

# Uncomment below if using LSTM or SARIMA
"""
windows = [7, 14, 30]

for feature in lag_features:
    for window in windows:
        df_clean[f'{feature}_ma_{window}'] = df_clean[feature].rolling(
            window=window, min_periods=1).mean()
        df_clean[f'{feature}_std_{window}'] = df_clean[feature].rolling(
            window=window, min_periods=1).std()

print(f"✓ Created {len(lag_features) * len(windows) * 2} rolling features")
print(f"  - Statistics: Mean, Std Dev")
print(f"  - Windows: {windows} days")
"""

### 3.5 One-Hot Encoding

In [ ]:
print("\n" + "="*80)
print("SEASONAL ENCODING - DUBAI PATTERN (NO MONTH DUMMIES, NO QUARTER)")
print("="*80)

# HIGH_SEASON and LOW_SEASON already created in Cell 29
# NO month dummies (redundant with High_Season/Low_Season)
# NO day-of-week encoding (not applicable for monthly data)
# NO Quarter (redundant with High_Season/Low_Season)

print("✓ Using Dubai Seasonality features (already created):")
print("  - High_Season: Oct-Apr (10,11,12,1,2,3,4) = 1")
print("  - Low_Season: May-Sep (5,6,7,8,9) = 1")
print("\n✓ SKIPPED: Month dummies (would add 12 redundant features)")
print("✓ SKIPPED: Day-of-week encoding (not applicable for monthly)")
print("✓ SKIPPED: Quarter (redundant - Q2 splits Dubai seasons)")

# Convert boolean to int (if any)
bool_cols = [col for col in df_clean.columns if df_clean[col].dtype == 'bool']
for col in bool_cols:
    df_clean[col] = df_clean[col].astype(int)

print(f"\nTotal temporal features: 2 (High_Season, Low_Season) - NO QUARTER!")

### 3.6 Chronological Data Splitting

In [ ]:
print("\n" + "="*80)
print("DATA SPLITTING (MONTHLY)")
print("="*80)

# Use FULL history (2009-2025) - monthly data benefits from longer history!
print("\n✅ Using FULL historical data (2009-2025) for monthly analysis")
print("  - Rationale: Monthly data less noisy than daily, more stable patterns")
print(f"Full dataset: {len(df_clean)} monthly records from {df_clean['Date'].min()} to {df_clean['Date'].max()}")

# Create splits - convert YYYY-MM strings to datetime for comparison
train_end_date = pd.to_datetime(TRAIN_END + '-01')
val_start_date = pd.to_datetime(VALIDATION_START + '-01')
val_end_date = pd.to_datetime(VALIDATION_END + '-01')
forecast_start_date = pd.to_datetime(FORECAST_START + '-01')
forecast_end_date = pd.to_datetime(FORECAST_END + '-01')

train_data = df_clean[df_clean['Date'] <= train_end_date].copy()
validation_data = df_clean[(df_clean['Date'] >= val_start_date) & 
                           (df_clean['Date'] <= val_end_date)].copy()
test_data = df_clean[(df_clean['Date'] >= forecast_start_date) & 
                     (df_clean['Date'] <= forecast_end_date)].copy()

print(f"\nTraining Set:")
print(f"  Period: {train_data['Date'].min()} to {train_data['Date'].max()}")
print(f"  Records: {len(train_data)} months")

print(f"\nValidation Set:")
print(f"  Period: {validation_data['Date'].min()} to {validation_data['Date'].max()}")
print(f"  Records: {len(validation_data)} months")

print(f"\nTest/Forecast Set:")
print(f"  Period: {test_data['Date'].min()} to {test_data['Date'].max()}")
print(f"  Records: {len(test_data)} months")

print(f"\nTotal: {len(df_clean)} monthly records")
print(f"✓ Sample-to-feature ratio (after removing NaN): ~{len(train_data)//6 if len(train_data) > 0 else 0}:1 (target: >10:1)")

### 3.7 Feature Scaling

In [ ]:
print("\n" + "="*80)
print("FEATURE SCALING (STANDARDIZATION - MONTHLY, NO QUARTER)")
print("="*80)

# Columns NOT to scale (binary/categorical only - NO QUARTER!)
cols_not_to_scale = ['Date', 'Month', 'High_Season', 'Low_Season']

# Get numeric columns to scale (targets + lags only)
numeric_cols_all = df_clean.select_dtypes(include=[np.number]).columns.tolist()
cols_to_scale = [col for col in numeric_cols_all if col not in cols_not_to_scale]

print(f"Columns NOT scaled: {len(cols_not_to_scale)} (categorical/binary)")
print(f"  {cols_not_to_scale}")
print(f"\nColumns TO scale: {len(cols_to_scale)} (continuous values)")
print(f"  {cols_to_scale}")

# Save ORIGINAL data for SARIMAX (statistical models use unscaled data)
train_data_original = train_data.copy()
validation_data_original = validation_data.copy()
test_data_original = test_data.copy()

# Fit scaler ONLY on training data
scaler = StandardScaler()
scaler.fit(train_data[cols_to_scale])

# Transform all datasets
train_data[cols_to_scale] = scaler.transform(train_data[cols_to_scale])
validation_data[cols_to_scale] = scaler.transform(validation_data[cols_to_scale])
test_data[cols_to_scale] = scaler.transform(test_data[cols_to_scale])

print("\n✓ StandardScaler fitted on training data ONLY")
print("✓ Applied to train, validation, and test sets")
print("✓ ORIGINAL data saved for SARIMAX model (train_data_original, etc.)")

### 3.8 Create ML-Ready Dataset

In [ ]:
print("\n" + "="*80)
print("ML-READY DATASET")
print("="*80)

# Combine all data
ml_ready_data = pd.concat([train_data, validation_data, test_data], ignore_index=True)

# Add dataset identifier
ml_ready_data['Dataset'] = 'Train'
ml_ready_data.loc[ml_ready_data['Date'] >= VALIDATION_START, 'Dataset'] = 'Validation'
ml_ready_data.loc[ml_ready_data['Date'] >= FORECAST_START, 'Dataset'] = 'Test'

# Drop redundant columns before export
redundant_cols = ['DOW', 'DayOfWeek_Name', 'Rm Sold', 'Year', 'Month_Name', 'Days', 'Total_Revenue', 'Total_Rooms_Sold']
cols_to_drop = [col for col in redundant_cols if col in ml_ready_data.columns]
if cols_to_drop:
    ml_ready_data = ml_ready_data.drop(columns=cols_to_drop)
    print(f"\n✓ Removed {len(cols_to_drop)} redundant columns: {cols_to_drop}")

print(f"\nTotal records: {len(ml_ready_data)}")
print(f"Total features: {len(ml_ready_data.columns)}")
print(f"Date range: {ml_ready_data['Date'].min()} to {ml_ready_data['Date'].max()}")

print(f"\nDataset Distribution:")
print(ml_ready_data['Dataset'].value_counts().sort_index())

print("\nSample of ML-Ready Data:")
# Show features that actually exist for monthly data
display_cols = ['Date', 'RevPar', 'High_Season', 'Low_Season', 'Dataset']
# Add lag feature if it exists
if 'RevPar_lag_1' in ml_ready_data.columns:
    display_cols.insert(2, 'RevPar_lag_1')
display(ml_ready_data[display_cols].head(10))

### 3.9 Export ML-Ready Data

In [ ]:
# Export to CSV
output_filename = 'combined_occupancy_ml_ready.csv'
ml_ready_data.to_csv(output_filename, index=False)

print("="*80)
print("EXPORT COMPLETE")
print("="*80)
print(f"\n✓ ML-ready data saved: {output_filename}")
print(f"✓ Ready for model training and forecasting")

# Download
files.download(output_filename)
print(f"\n✓ File downloaded!")

---
## Preprocessing Summary

**Completed Steps:**
1. ✓ Data cleaning and missing value imputation
2. ✓ Temporal features (Day of Week, Month, Quarter, etc.)
3. ✓ Lagged variables (1-day lag only)
4. ✓ Moving averages: REMOVED (not needed for XGBoost)
5. ✓ One-hot encoding (DOW, Month)
6. ✓ Chronological data splitting (Train/Validation/Test)
7. ✓ Feature scaling (StandardScaler fitted on training only)
8. ✓ Export ML-ready dataset

**Next Steps:**
- Model training (XGBoost, SARIMA, LSTM)
- Model evaluation on validation set
- Final forecast: September - December 2025

---
## Section 4: Model Training and Selection

Train top-ranked models using the prepared training data.

### 4.0 Load Preprocessed ML-Ready Data

In [ ]:
# If you already have ml_ready_data from Section 3, skip this cell
# Otherwise, upload the preprocessed file

print("Upload combined_occupancy_ml_ready.csv if not already in memory:")
try:
    # Check if ml_ready_data exists
    print(f"ML-ready data already loaded: {ml_ready_data.shape}")
except NameError:
    # Upload and load the preprocessed file
    uploaded_ml = files.upload()
    ml_filename = list(uploaded_ml.keys())[0]
    ml_ready_data = pd.read_csv(ml_filename)
    ml_ready_data['Date'] = pd.to_datetime(ml_ready_data['Date'])
    print(f"ML-ready data loaded: {ml_ready_data.shape}")

In [ ]:
# Prepare datasets for modeling
print("="*80)
print("PREPARING DATA FOR MODELING (MONTHLY - 6 FEATURES, NO QUARTER)")
print("="*80)

# Split by Dataset identifier
train_df = ml_ready_data[ml_ready_data['Dataset'] == 'Train'].copy()
val_df = ml_ready_data[ml_ready_data['Dataset'] == 'Validation'].copy()
test_df = ml_ready_data[ml_ready_data['Dataset'] == 'Test'].copy()

print(f"\nTraining Set: {len(train_df)} months")
print(f"Validation Set: {len(val_df)} months")
print(f"Test Set: {len(test_df)} months")

# Define feature columns (6 features - NO QUARTER!)
exclude_cols = ['Date', 'Month', 'Dataset'] + TARGET_VARIABLES
feature_cols = [col for col in ml_ready_data.columns if col not in exclude_cols]

# Remove any columns with NaN (from lagging)
train_df_clean = train_df.dropna()
val_df_clean = val_df.dropna()

print(f"\nFeature columns: {len(feature_cols)}")
print(f"  {feature_cols}")
print(f"\nExpected: 6 features [High_Season, Low_Season, RevPar_lag_1, ADR_lag_1, Revenue_lag_1, Occupancy_Pct_lag_1]")

print(f"\nTraining set after removing NaN: {len(train_df_clean)} months")
print(f"Validation set after removing NaN: {len(val_df_clean)} months")
print(f"\nSample-to-feature ratio: {len(train_df_clean)}:{len(feature_cols)} = {len(train_df_clean)//len(feature_cols) if len(feature_cols) > 0 else 0}:1")

### 4.0.1 Important Note: Data Scaling Strategy

**Different models require different data scales:**

1. **SARIMA (Statistical Model)**
   - Trains on **ORIGINAL (unscaled)** data
   - Statistical models expect data in its natural scale
   - Predictions are directly in AED (currency units)
   - Better performance when working with actual values

2. **XGBoost (Tree-based Model)**
   - Trains on **SCALED** data
   - Benefits from standardization for regularization
   - Predictions need inverse transformation back to AED

3. **LSTM (Neural Network)**
   - Trains on **SCALED** data  
   - Neural networks require normalized inputs for optimization
   - Predictions need inverse transformation back to AED

**Why this matters:**
- SARIMA was performing poorly when trained on scaled data
- Each model type has different requirements for optimal performance
- We maintain separate datasets to ensure each model gets the appropriate scale

### 4.1 Baseline Model: SARIMA

Establish a performance benchmark using SARIMA (Seasonal ARIMA).

In [ ]:
# Import SARIMAX libraries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("="*80)
print("BASELINE MODEL: SARIMAX (WITH DUBAI SEASONALITY)")
print("="*80)

# SARIMAX uses ORIGINAL (unscaled) data + exogenous variables
train_sarimax = train_data_original.set_index('Date')['RevPar']
# Ensure High_Season and Low_Season exist in the training data
if 'High_Season' not in train_data_original.columns:
    train_data_original['High_Season'] = train_data_original['Month'].isin([10,11,12,1,2,3,4]).astype(int)
    train_data_original['Low_Season'] = train_data_original['Month'].isin([5,6,7,8,9]).astype(int)
    
train_exog = train_data_original.set_index('Date')[['High_Season', 'Low_Season']]

val_sarimax = validation_data_original.set_index('Date')['RevPar']
if 'High_Season' not in validation_data_original.columns:
    validation_data_original['High_Season'] = validation_data_original['Month'].isin([10,11,12,1,2,3,4]).astype(int)
    validation_data_original['Low_Season'] = validation_data_original['Month'].isin([5,6,7,8,9]).astype(int)
    
val_exog = validation_data_original.set_index('Date')[['High_Season', 'Low_Season']]

print(f"\nSARIMAX Training Data (Original Scale):")
print(f"  Records: {len(train_sarimax)} months")
print(f"  RevPar range: AED {train_sarimax.min():.2f} - {train_sarimax.max():.2f}")
print(f"  Mean: AED {train_sarimax.mean():.2f}")
print(f"  Exogenous vars: High_Season, Low_Season (Dubai pattern)")

# Train SARIMAX model with Dubai seasonality
sarimax_model = SARIMAX(
    train_sarimax,
    exog=train_exog,  # EXOGENOUS VARIABLES!
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 12),  # s=12 for monthly!
    enforce_stationarity=False,
    enforce_invertibility=False
)

print("\nTraining SARIMAX with Dubai seasonality exogenous variables...")
sarimax_fit = sarimax_model.fit(disp=False, maxiter=200)
print("✓ SARIMAX model trained on ORIGINAL (unscaled) data")

# Predict with exogenous variables
sarimax_val_pred = sarimax_fit.forecast(steps=len(val_sarimax), exog=val_exog.values)

# Calculate metrics
sarimax_rmse = np.sqrt(mean_squared_error(val_sarimax, sarimax_val_pred))
sarimax_mae = mean_absolute_error(val_sarimax, sarimax_val_pred)
sarimax_r2 = r2_score(val_sarimax, sarimax_val_pred)
sarimax_mape = np.mean(np.abs((val_sarimax - sarimax_val_pred) / val_sarimax)) * 100

print(f"\nSARIMAX Validation Performance (Original Scale):")
print(f"  RMSE: AED {sarimax_rmse:.2f}")
print(f"  MAE: AED {sarimax_mae:.2f}")
print(f"  R²: {sarimax_r2:.4f}")
print(f"  MAPE: {sarimax_mape:.2f}%")

print(f"\n✓ SARIMAX predictions are in ORIGINAL SCALE (AED)")
print(f"  Prediction range: AED {sarimax_val_pred.min():.2f} - {sarimax_val_pred.max():.2f}")

# Store results
results = {
    'SARIMAX': {'RMSE': sarimax_rmse, 'MAE': sarimax_mae, 'R2': sarimax_r2, 'MAPE': sarimax_mape}
}

In [ ]:
# This cell has been removed - duplicate SARIMA cell
# SARIMAX with Dubai seasonality is implemented in Cell 50

### 4.2 Advanced Model: XGBoost Regressor

Train XGBoost for multi-output regression.

In [ ]:
# Import XGBoost
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

print("\n" + "="*80)
print("ADVANCED MODEL: XGBOOST REGRESSOR (MONTHLY - REDUCED COMPLEXITY)")
print("="*80)

# Prepare data
X_train = train_df_clean[feature_cols]
y_train = train_df_clean[TARGET_VARIABLES]

X_val = val_df_clean[feature_cols]
y_val = val_df_clean[TARGET_VARIABLES]

print(f"\nTraining features shape: {X_train.shape}")
print(f"Training targets shape: {y_train.shape}")

# MONTHLY: Reduced complexity for ~200 monthly records
xgb_model = xgb.XGBRegressor(
    n_estimators=100,  # REDUCED from 200 for monthly
    max_depth=4,  # REDUCED from 5 for monthly
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.7,
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=1.0,  # L2 regularization
    min_child_weight=3,  # Prevents overfitting
    random_state=42,
    n_jobs=-1
)

# Use MultiOutputRegressor for multiple targets
multi_xgb = MultiOutputRegressor(xgb_model)

print("\nTraining XGBoost model (monthly-optimized complexity)...")
multi_xgb.fit(X_train, y_train)
print("✓ XGBoost model trained")

# Predict on validation set
xgb_val_pred = multi_xgb.predict(X_val)

# FIXED: Inverse transform predictions and actuals to original scale
xgb_val_pred_original = xgb_val_pred.copy()
y_val_original = y_val.copy()

# Inverse transform each target column
for idx_col, target in enumerate(TARGET_VARIABLES):
    if target in cols_to_scale:
        # Get the scaler index for this target
        target_idx = cols_to_scale.index(target)
        
        # Inverse transform predictions
        dummy_pred = np.zeros((len(xgb_val_pred), len(cols_to_scale)))
        dummy_pred[:, target_idx] = xgb_val_pred[:, idx_col]
        inv_transformed_pred = scaler.inverse_transform(dummy_pred)
        xgb_val_pred_original[:, idx_col] = inv_transformed_pred[:, target_idx]
        
        # Inverse transform actuals
        dummy_actual = np.zeros((len(y_val), len(cols_to_scale)))
        dummy_actual[:, target_idx] = y_val.iloc[:, idx_col]
        inv_transformed_actual = scaler.inverse_transform(dummy_actual)
        y_val_original.iloc[:, idx_col] = inv_transformed_actual[:, target_idx]

# Calculate metrics for each target (on scaled data for consistency)
print(f"\nXGBoost Validation Performance (Scaled Data):")
xgb_metrics = {}
for idx, target in enumerate(TARGET_VARIABLES):
    rmse = np.sqrt(mean_squared_error(y_val.iloc[:, idx], xgb_val_pred[:, idx]))
    mae = mean_absolute_error(y_val.iloc[:, idx], xgb_val_pred[:, idx])
    r2 = r2_score(y_val.iloc[:, idx], xgb_val_pred[:, idx])
    
    xgb_metrics[target] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
    print(f"\n  {target}:")
    print(f"    RMSE: {rmse:.2f}")
    print(f"    MAE: {mae:.2f}")
    print(f"    R²: {r2:.4f}")

results['XGBoost'] = xgb_metrics

### 4.2.1 Validation Set: Actual vs Predicted Comparison

Compare predictions against actual values (June-August 2025) in original scale.

In [ ]:
print("\n" + "="*80)
print("VALIDATION SET: ACTUAL VS PREDICTED (ORIGINAL SCALE)")
print("="*80)

# Recalculate metrics on original scale
print(f"\nXGBoost Validation Performance (Original Scale):")
xgb_metrics_original = {}
for idx_col, target in enumerate(TARGET_VARIABLES):
    actual = y_val_original.iloc[:, idx_col]
    predicted = xgb_val_pred_original[:, idx_col]
    
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    
    # Calculate MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    xgb_metrics_original[target] = {
        'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape
    }
    
    print(f"\n  {target}:")
    print(f"    RMSE: AED {rmse:,.2f}" if 'Revenue' in target or 'ADR' in target or 'RevPar' in target else f"    RMSE: {rmse:.2f}%")
    print(f"    MAE: AED {mae:,.2f}" if 'Revenue' in target or 'ADR' in target or 'RevPar' in target else f"    MAE: {mae:.2f}%")
    print(f"    R²: {r2:.4f}")
    print(f"    MAPE: {mape:.2f}%")

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Date': val_df_clean['Date'].values,
})

for idx_col, target in enumerate(TARGET_VARIABLES):
    comparison_df[f'{target}_Actual'] = y_val_original.iloc[:, idx_col].values
    comparison_df[f'{target}_Predicted'] = xgb_val_pred_original[:, idx_col]
    comparison_df[f'{target}_Error'] = comparison_df[f'{target}_Actual'] - comparison_df[f'{target}_Predicted']
    comparison_df[f'{target}_Error_Pct'] = (comparison_df[f'{target}_Error'] / comparison_df[f'{target}_Actual']) * 100

print("\n" + "="*80)
print("VALIDATION COMPARISON SAMPLE (First 10 days)")
print("="*80)
display(comparison_df[['Date', 'RevPar_Actual', 'RevPar_Predicted', 'RevPar_Error', 'RevPar_Error_Pct']].head(10))

# Summary statistics
print("\n" + "="*80)
print("VALIDATION PERIOD SUMMARY")
print("="*80)
for target in TARGET_VARIABLES:
    actual_total = comparison_df[f'{target}_Actual'].sum()
    predicted_total = comparison_df[f'{target}_Predicted'].sum()
    diff = predicted_total - actual_total
    diff_pct = (diff / actual_total) * 100
    
    print(f"\n{target}:")
    if 'Revenue' in target:
        print(f"  Actual Total: AED {actual_total:,.2f}")
        print(f"  Predicted Total: AED {predicted_total:,.2f}")
        print(f"  Difference: AED {diff:,.2f} ({diff_pct:+.2f}%)")
    else:
        print(f"  Actual Average: {comparison_df[f'{target}_Actual'].mean():.2f}")
        print(f"  Predicted Average: {comparison_df[f'{target}_Predicted'].mean():.2f}")
        print(f"  Difference: {comparison_df[f'{target}_Predicted'].mean() - comparison_df[f'{target}_Actual'].mean():+.2f}")


### 4.3 Advanced Model: LSTM Neural Network

Train LSTM for sequential time series prediction.

In [ ]:
# Import TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

print("\n" + "="*80)
print("ADVANCED MODEL: LSTM NEURAL NETWORK (MONTHLY)")
print("="*80)

# Prepare data for LSTM (reshape to 3D: samples, timesteps, features)
def create_sequences(X, y, timesteps=6):
    Xs, ys = [], []
    for i in range(len(X) - timesteps):
        Xs.append(X.iloc[i:(i + timesteps)].values)
        ys.append(y.iloc[i + timesteps].values)
    return np.array(Xs), np.array(ys)

timesteps = 6  # Use 6 MONTHS of history (not 7 days!)

# FIXED: Ensure all data is float32 for TensorFlow compatibility
X_train_float32 = X_train.astype(np.float32)
y_train_float32 = y_train.astype(np.float32)
X_val_float32 = X_val.astype(np.float32)
y_val_float32 = y_val.astype(np.float32)

X_train_lstm, y_train_lstm = create_sequences(X_train_float32, y_train_float32, timesteps)
X_val_lstm, y_val_lstm = create_sequences(X_val_float32, y_val_float32, timesteps)

print(f"\nLSTM Training data shape: {X_train_lstm.shape}")
print(f"LSTM Validation data shape: {X_val_lstm.shape}")

# Build LSTM model - MONTHLY optimized (reduced complexity)
lstm_model = Sequential([
    LSTM(24, activation='tanh', return_sequences=False,  # REDUCED from 32 for monthly
         input_shape=(timesteps, X_train.shape[1])),
    Dropout(0.3),
    Dense(12, activation='relu'),  # REDUCED from 16 for monthly
    Dense(len(TARGET_VARIABLES))  # Output layer for all targets
])

# Use lower learning rate to prevent overfitting
lstm_model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])

print("\nLSTM Model Architecture (Monthly-Optimized):")
lstm_model.summary()

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train LSTM with monthly-optimized settings
print("\nTraining LSTM model with monthly-optimized complexity...")
history = lstm_model.fit(
    X_train_lstm, y_train_lstm,
    validation_data=(X_val_lstm, y_val_lstm),
    epochs=50,
    batch_size=16,
    callbacks=[early_stop],
    verbose=0
)
print("✓ LSTM model trained")

# Predict on validation set
lstm_val_pred = lstm_model.predict(X_val_lstm, verbose=0)

# Calculate metrics
print(f"\nLSTM Validation Performance:")
lstm_metrics = {}
for idx, target in enumerate(TARGET_VARIABLES):
    rmse = np.sqrt(mean_squared_error(y_val_lstm[:, idx], lstm_val_pred[:, idx]))
    mae = mean_absolute_error(y_val_lstm[:, idx], lstm_val_pred[:, idx]))
    r2 = r2_score(y_val_lstm[:, idx], lstm_val_pred[:, idx])
    
    lstm_metrics[target] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
    print(f"\n  {target}:")
    print(f"    RMSE: {rmse:.2f}")
    print(f"    MAE: {mae:.2f}")
    print(f"    R²: {r2:.4f}")

results['LSTM'] = lstm_metrics

### 4.4 Training Loss Visualization (LSTM)

In [ ]:
# Plot LSTM training history
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_title('LSTM Model Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss (MSE)', fontsize=11)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_title('LSTM Model MAE', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('MAE', fontsize=11)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("="*80)
print("MODEL COMPARISON - VALIDATION SET")
print("="*80)

# IMPORTANT NOTE: 
# - SARIMA metrics are in ORIGINAL SCALE (AED)
# - XGBoost and LSTM metrics are in SCALED units
# For fair comparison, we compare them separately or convert to same scale

print("\n" + "="*80)
print("SARIMA Performance (Original Scale - AED)")
print("="*80)
print(f"\nRevPar Prediction:")
print(f"  RMSE: AED {results['SARIMA']['RMSE']:.2f}")
print(f"  MAE: AED {results['SARIMA']['MAE']:.2f}")
print(f"  R²: {results['SARIMA']['R2']:.4f}")
print(f"  MAPE: {results['SARIMA']['MAPE']:.2f}%")

print("\n" + "="*80)
print("XGBoost Performance (Original Scale - from Section 4.2.1)")
print("="*80)
# Use the metrics calculated in original scale from cell 53
for target in TARGET_VARIABLES:
    if target in xgb_metrics_original:
        print(f"\n{target}:")
        metrics = xgb_metrics_original[target]
        if 'Revenue' in target or 'ADR' in target or 'RevPar' in target:
            print(f"  RMSE: AED {metrics['RMSE']:,.2f}")
            print(f"  MAE: AED {metrics['MAE']:,.2f}")
        else:
            print(f"  RMSE: {metrics['RMSE']:.2f}%")
            print(f"  MAE: {metrics['MAE']:.2f}%")
        print(f"  R²: {metrics['R2']:.4f}")
        print(f"  MAPE: {metrics['MAPE']:.2f}%")

print("\n" + "="*80)
print("LSTM Performance (Scaled Data)")
print("="*80)
print("Note: LSTM metrics are in scaled units (not directly comparable to SARIMA)")
for target in TARGET_VARIABLES:
    print(f"\n{target}:")
    print(f"  RMSE: {results['LSTM'][target]['RMSE']:.2f} (scaled)")
    print(f"  MAE: {results['LSTM'][target]['MAE']:.2f} (scaled)")
    print(f"  R²: {results['LSTM'][target]['R2']:.4f}")

# Create comparison visualization (RevPar only, original scale)
print("\n" + "="*80)
print("VISUAL COMPARISON - RevPar (Original Scale)")
print("="*80)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Compare SARIMA vs XGBoost (both in original scale)
models_to_compare = ['SARIMA', 'XGBoost']
metrics_to_plot = ['RMSE', 'MAE', 'MAPE']
metric_values = {
    'SARIMA': [
        results['SARIMA']['RMSE'],
        results['SARIMA']['MAE'],
        results['SARIMA']['MAPE']
    ],
    'XGBoost': [
        xgb_metrics_original['RevPar']['RMSE'],
        xgb_metrics_original['RevPar']['MAE'],
        xgb_metrics_original['RevPar']['MAPE']
    ]
}

for idx, metric in enumerate(metrics_to_plot):
    values = [metric_values[model][idx] for model in models_to_compare]
    axes[idx].bar(models_to_compare, values, color=['skyblue', 'orange'])
    
    if metric == 'MAPE':
        axes[idx].set_title(f'{metric} Comparison - RevPar (%)', fontsize=14, fontweight='bold')
        axes[idx].set_ylabel(f'{metric} (%)', fontsize=11)
    else:
        axes[idx].set_title(f'{metric} Comparison - RevPar (AED)', fontsize=14, fontweight='bold')
        axes[idx].set_ylabel(f'{metric} (AED)', fontsize=11)
    
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, v in enumerate(values):
        axes[idx].text(i, v, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("MODEL SELECTION SUMMARY")
print("="*80)
print("\nKey Findings:")
print(f"  - SARIMA trained on ORIGINAL scale performs better for statistical modeling")
print(f"  - XGBoost trained on SCALED data benefits from regularization")
print(f"  - LSTM trained on SCALED data for neural network optimization")
print(f"\nRecommendation: Use XGBoost for final forecast (multi-target, best R²)")

In [ ]:
print("="*80)
print("MODEL COMPARISON - VALIDATION SET")
print("="*80)

# Create comparison dataframe
comparison_data = []

# SARIMA (RevPar only)
comparison_data.append({
    'Model': 'SARIMA',
    'Target': 'RevPar',
    'RMSE': results['SARIMA']['RMSE'],
    'MAE': results['SARIMA']['MAE'],
    'R²': results['SARIMA']['R2']
})

# XGBoost
for target in TARGET_VARIABLES:
    comparison_data.append({
        'Model': 'XGBoost',
        'Target': target,
        'RMSE': results['XGBoost'][target]['RMSE'],
        'MAE': results['XGBoost'][target]['MAE'],
        'R²': results['XGBoost'][target]['R2']
    })

# LSTM
for target in TARGET_VARIABLES:
    comparison_data.append({
        'Model': 'LSTM',
        'Target': target,
        'RMSE': results['LSTM'][target]['RMSE'],
        'MAE': results['LSTM'][target]['MAE'],
        'R²': results['LSTM'][target]['R2']
    })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['RMSE', 'MAE', 'R²']
for idx, metric in enumerate(metrics):
    revpar_data = comparison_df[comparison_df['Target'] == 'RevPar']
    axes[idx].bar(revpar_data['Model'], revpar_data[metric], 
                  color=['skyblue', 'orange', 'green'])
    axes[idx].set_title(f'{metric} Comparison (RevPar)', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel(metric, fontsize=11)
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 5.2 Final Model Training (All Historical Data)

Retrain the best model using all available data up to August 29, 2025.

In [ ]:
print("\n" + "="*80)
print("FINAL MODEL TRAINING (TRAIN + VALIDATION)")
print("="*80)

# Combine train and validation for final training
final_train_df = pd.concat([train_df_clean, val_df_clean]).reset_index(drop=True)

X_final = final_train_df[feature_cols]
y_final = final_train_df[TARGET_VARIABLES]

print(f"\nFinal training set: {X_final.shape[0]} records")
print(f"Date range: {final_train_df['Date'].min()} to {final_train_df['Date'].max()}")

# Retrain XGBoost with improved regularization parameters
print("\nRetraining XGBoost on all historical data with regularization...")
final_xgb = MultiOutputRegressor(xgb.XGBRegressor(
    n_estimators=200,
    max_depth=5,  # Reduced from 6
    learning_rate=0.05,  # Reduced from 0.1
    subsample=0.8,
    colsample_bytree=0.7,  # Reduced from 0.8
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=1.0,  # L2 regularization
    min_child_weight=3,  # Prevents overfitting
    random_state=42,
    n_jobs=-1
))

final_xgb.fit(X_final, y_final)
print("✓ Final XGBoost model trained with regularization")

### 5.3 Forecasting the Future (Sept - Dec 2025)

In [ ]:
print("\n" + "="*80)
print("FINAL FORECAST: SEPTEMBER - DECEMBER 2025")
print("="*80)

# Prepare test data (handle NaN from lagging)
# For forecasting, we need to handle missing lag values
# Use forward fill or last known values
test_df_forecast = test_df.copy()

# Fill NaN with forward fill (use last known values)
for col in feature_cols:
    if test_df_forecast[col].isnull().any():
        # Fill with the last value from validation set
        last_val = val_df_clean[col].iloc[-1] if col in val_df_clean.columns else 0
        test_df_forecast[col].fillna(last_val, inplace=True)

X_test = test_df_forecast[feature_cols]

print(f"\nTest set for forecasting: {X_test.shape[0]} records")
print(f"Date range: {test_df_forecast['Date'].min()} to {test_df_forecast['Date'].max()}")

# Make predictions
final_predictions = final_xgb.predict(X_test)

# Inverse transform predictions to original scale
final_predictions_original = final_predictions.copy()
for idx_col, target in enumerate(TARGET_VARIABLES):
    if target in cols_to_scale:
        target_idx = cols_to_scale.index(target)
        dummy = np.zeros((len(final_predictions), len(cols_to_scale)))
        dummy[:, target_idx] = final_predictions[:, idx_col]
        inv_transformed = scaler.inverse_transform(dummy)
        final_predictions_original[:, idx_col] = inv_transformed[:, target_idx]

# Create forecast dataframe with original scale values
forecast_df = pd.DataFrame(final_predictions_original, columns=TARGET_VARIABLES)

# Create forecast dataframe
forecast_df['Date'] = test_df_forecast['Date'].values

print("\n✓ Forecast complete!")
print("\nForecast Summary (Sept - Dec 2025):")
display(forecast_df.head(10))

In [ ]:
# DIAGNOSTIC: Check if forecast values are reasonable
print("\n" + "="*80)
print("FORECAST DIAGNOSTICS")
print("="*80)

print(f"\nForecast Statistics:")
print(f"  Revenue - Min: AED {forecast_df['Revenue'].min():,.2f}, Max: AED {forecast_df['Revenue'].max():,.2f}")
print(f"  RevPar - Min: AED {forecast_df['RevPar'].min():.2f}, Max: AED {forecast_df['RevPar'].max():.2f}")
print(f"  ADR - Min: AED {forecast_df['ADR'].min():.2f}, Max: AED {forecast_df['ADR'].max():.2f}")
print(f"  Occupancy - Min: {forecast_df['Occupancy_Pct'].min():.2f}%, Max: {forecast_df['Occupancy_Pct'].max():.2f}%")

print(f"\nHistorical Comparison (2024 Sept-Dec for reference):")
hist_comp = df_clean[(df_clean['Date'] >= '2024-09-01') & (df_clean['Date'] <= '2024-12-31')]
if len(hist_comp) > 0:
    print(f"  2024 Sept-Dec Revenue: AED {hist_comp['Revenue'].sum():,.2f}")
    print(f"  2025 Forecast: AED {forecast_df['Revenue'].sum():,.2f}")
    diff_pct = ((forecast_df['Revenue'].sum() / hist_comp['Revenue'].sum()) - 1) * 100
    print(f"  Difference: {diff_pct:+.1f}%")
else:
    print("  No 2024 Sept-Dec data available for comparison")

### 5.4 Final Visualization and Total Revenue Projection

In [ ]:
print("\n" + "="*80)
print("TOTAL REVENUE PROJECTION (SEPT - DEC 2025)")
print("="*80)

# Calculate totals
total_revenue = forecast_df['Revenue'].sum()
avg_revpar = forecast_df['RevPar'].mean()
avg_adr = forecast_df['ADR'].mean()
avg_occupancy = forecast_df['Occupancy_Pct'].mean()

print(f"\nProjected Totals (September 1 - December 31, 2025):")
print(f"  Total Revenue: AED {total_revenue:,.2f}")
print(f"  Average RevPar: AED {avg_revpar:.2f}")
print(f"  Average ADR: AED {avg_adr:.2f}")
print(f"  Average Occupancy: {avg_occupancy:.2f}%")

# Monthly breakdown
forecast_df['Month'] = pd.to_datetime(forecast_df['Date']).dt.month
monthly_summary = forecast_df.groupby('Month').agg({
    'Revenue': 'sum',
    'RevPar': 'mean',
    'ADR': 'mean',
    'Occupancy_Pct': 'mean'
})

print("\nMonthly Breakdown:")
monthly_summary.index = ['September', 'October', 'November', 'December']
display(monthly_summary)

### 5.4.1 Full Year 2025 Summary (Actual + Forecasted)

Combine actual data (Jan-Aug) with forecast (Sept-Dec) for complete year view.

In [ ]:
print("\n" + "="*80)
print("FULL YEAR 2025 SUMMARY (ACTUAL + FORECASTED)")
print("="*80)

# FIXED: Get actual data for Jan-Aug 2025 and INVERSE TRANSFORM
actual_2025_scaled = ml_ready_data[(ml_ready_data['Date'] >= '2025-01-01') &
                                    (ml_ready_data['Date'] <= pd.to_datetime('2025-08-31'))].copy()

# The data in ml_ready_data is SCALED, so we need to inverse transform it
# Create a copy for inverse transformation
actual_2025_original = actual_2025_scaled.copy()

# Inverse transform each target variable
for target in TARGET_VARIABLES:
    if target in cols_to_scale:
        target_idx = cols_to_scale.index(target)
        
        # Get scaled values
        scaled_values = actual_2025_scaled[target].values
        
        # Create dummy array for inverse transform
        dummy = np.zeros((len(scaled_values), len(cols_to_scale)))
        dummy[:, target_idx] = scaled_values
        
        # Inverse transform
        inv_transformed = scaler.inverse_transform(dummy)
        actual_2025_original[target] = inv_transformed[:, target_idx]

# Now calculate metrics from ORIGINAL scale data
actual_revenue_ytd = actual_2025_original['Revenue'].sum()
actual_avg_revpar = actual_2025_original['RevPar'].mean()
actual_avg_adr = actual_2025_original['ADR'].mean()
actual_avg_occ = actual_2025_original['Occupancy_Pct'].mean()

# Forecast totals (Sept-Dec)
forecast_revenue = forecast_df['Revenue'].sum()
forecast_avg_revpar = forecast_df['RevPar'].mean()
forecast_avg_adr = forecast_df['ADR'].mean()
forecast_avg_occ = forecast_df['Occupancy_Pct'].mean()

# Full year totals
full_year_revenue = actual_revenue_ytd + forecast_revenue
full_year_avg_revpar = (actual_avg_revpar * len(actual_2025_original) + forecast_avg_revpar * len(forecast_df)) / (len(actual_2025_original) + len(forecast_df))
full_year_avg_adr = (actual_avg_adr * len(actual_2025_original) + forecast_avg_adr * len(forecast_df)) / (len(actual_2025_original) + len(forecast_df))
full_year_avg_occ = (actual_avg_occ * len(actual_2025_original) + forecast_avg_occ * len(forecast_df)) / (len(actual_2025_original) + len(forecast_df))

print(f"\n2025 Year-to-Date (Jan - Aug): ACTUAL")
print(f"  Months: {len(actual_2025_original)}")
print(f"  Total Revenue: AED {actual_revenue_ytd:,.2f}")
print(f"  Average RevPar: AED {actual_avg_revpar:,.2f}")
print(f"  Average ADR: AED {actual_avg_adr:,.2f}")
print(f"  Average Occupancy: {actual_avg_occ:.2f}%")

print(f"\n2025 Forecast (Sept - Dec): FORECASTED")
print(f"  Months: {len(forecast_df)}")
print(f"  Total Revenue: AED {forecast_revenue:,.2f}")
print(f"  Average RevPar: AED {forecast_avg_revpar:,.2f}")
print(f"  Average ADR: AED {forecast_avg_adr:,.2f}")
print(f"  Average Occupancy: {forecast_avg_occ:.2f}%")

print("\n" + "="*80)
print("FULL YEAR 2025 TOTALS (Jan - Dec)")
print("="*80)
print(f"  Total Months: {len(actual_2025_original) + len(forecast_df)}")
print(f"  Total Revenue: AED {full_year_revenue:,.2f}")
print(f"  Average RevPar: AED {full_year_avg_revpar:,.2f}")
print(f"  Average ADR: AED {full_year_avg_adr:,.2f}")
print(f"  Average Occupancy: {full_year_avg_occ:.2f}%")

print("\n" + "="*80)
print("BREAKDOWN BY PERIOD")
print("="*80)

# Create comparison table
summary_table = pd.DataFrame({
    'Period': ['Jan-Aug (Actual)', 'Sept-Dec (Forecast)', 'Full Year 2025'],
    'Months': [len(actual_2025_original), len(forecast_df), len(actual_2025_original) + len(forecast_df)],
    'Total Revenue (AED)': [
        f'{actual_revenue_ytd:,.2f}',
        f'{forecast_revenue:,.2f}',
        f'{full_year_revenue:,.2f}'
    ],
    'Avg RevPar (AED)': [
        f'{actual_avg_revpar:,.2f}',
        f'{forecast_avg_revpar:,.2f}',
        f'{full_year_avg_revpar:,.2f}'
    ],
    'Avg ADR (AED)': [
        f'{actual_avg_adr:,.2f}',
        f'{forecast_avg_adr:,.2f}',
        f'{full_year_avg_adr:,.2f}'
    ],
    'Avg Occupancy (%)': [
        f'{actual_avg_occ:.2f}',
        f'{forecast_avg_occ:.2f}',
        f'{full_year_avg_occ:.2f}'
    ]
})

display(summary_table)

# Monthly breakdown for full year
print("\n" + "="*80)
print("MONTHLY BREAKDOWN - 2025")
print("="*80)

# Get monthly data for actual
actual_2025_original['Month_Num'] = pd.to_datetime(actual_2025_original['Date']).dt.month
actual_monthly = actual_2025_original.groupby('Month_Num').agg({
    'Revenue': 'sum',
    'RevPar': 'mean',
    'ADR': 'mean',
    'Occupancy_Pct': 'mean'
})

# Get monthly data for forecast
forecast_df['Month_Num'] = pd.to_datetime(forecast_df['Date']).dt.month
forecast_monthly = forecast_df.groupby('Month_Num').agg({
    'Revenue': 'sum',
    'RevPar': 'mean',
    'ADR': 'mean',
    'Occupancy_Pct': 'mean'
})

# Combine
import calendar
monthly_full = pd.concat([actual_monthly, forecast_monthly]).sort_index()
monthly_full.index = [calendar.month_name[i] for i in monthly_full.index]

# Format for display
monthly_display = monthly_full.copy()
monthly_display['Type'] = ['Actual']*len(actual_monthly) + ['Forecast']*len(forecast_monthly)
monthly_display = monthly_display[['Type', 'Revenue', 'RevPar', 'ADR', 'Occupancy_Pct']]
monthly_display.columns = ['Type', 'Revenue (AED)', 'RevPar (AED)', 'ADR (AED)', 'Occupancy (%)']

display(monthly_display)

print(f"\n{'='*80}")
print(f"2025 ANNUAL REVENUE: AED {full_year_revenue:,.2f}")
print(f"{'='*80}")


In [ ]:
# Visualization: Historical + Forecast
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
axes = axes.ravel()

# Combine historical and forecast data for plotting
historical_data = ml_ready_data[ml_ready_data['Date'] <= VALIDATION_END].copy()

for idx, var in enumerate(TARGET_VARIABLES):
    # Plot historical data
    axes[idx].plot(historical_data['Date'], historical_data[var], 
                   linewidth=1.5, color='blue', label='Historical (Actual)', alpha=0.7)
    
    # Plot forecast
    axes[idx].plot(forecast_df['Date'], forecast_df[var], 
                   linewidth=2, color='red', label='Forecast (Sept-Dec 2025)', marker='o')
    
    # Add vertical lines
    axes[idx].axvline(x=pd.to_datetime(TRAIN_END), color='green', 
                      linestyle='--', alpha=0.5, label='Train End')
    axes[idx].axvline(x=pd.to_datetime(VALIDATION_END), color='orange', 
                      linestyle='--', alpha=0.5, label='Validation End')
    
    axes[idx].set_title(f'{var} - Historical vs Forecast', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=11)
    axes[idx].set_ylabel(var, fontsize=11)
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.5 Model Diagnostics: Feature Importance (XGBoost)

In [ ]:
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS (XGBOOST)")
print("="*80)

# Get feature importance for RevPar model (first target)
revpar_model = final_xgb.estimators_[0]  # First estimator is for RevPar
feature_importance = revpar_model.feature_importances_

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

# Plot top 20 features
top_n = 20
plt.figure(figsize=(12, 8))
plt.barh(range(top_n), importance_df['Importance'].head(top_n), color='steelblue')
plt.yticks(range(top_n), importance_df['Feature'].head(top_n))
plt.xlabel('Importance', fontsize=12)
plt.title(f'Top {top_n} Feature Importance (RevPar Prediction)', 
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
display(importance_df.head(10))

### 5.6 Residual Analysis (Validation Set)

In [ ]:
# Calculate residuals on validation set
residuals = y_val.values - xgb_val_pred

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.ravel()

for idx, target in enumerate(TARGET_VARIABLES):
    axes[idx].scatter(val_df_clean['Date'].values, residuals[:, idx], 
                     alpha=0.6, color='purple')
    axes[idx].axhline(y=0, color='red', linestyle='--', linewidth=2)
    axes[idx].set_title(f'{target} - Residuals vs Time (Validation)', 
                       fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=11)
    axes[idx].set_ylabel('Residuals', fontsize=11)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.7 Export Final Forecast

In [ ]:
# Export forecast to CSV
forecast_output = 'hotel_revenue_forecast_sept_dec_2025.csv'
forecast_df.to_csv(forecast_output, index=False)

print("="*80)
print("FORECAST EXPORT COMPLETE")
print("="*80)
print(f"\n✓ Forecast saved to: {forecast_output}")

# Download
files.download(forecast_output)
print("\n✓ File downloaded!")

print("\n" + "="*80)
print("PROJECT COMPLETE!")
print("="*80)
print(f"\nFinal 2025 Revenue Projection (Sept-Dec): AED {total_revenue:,.2f}")